<a href="https://colab.research.google.com/github/slucasmyer/rl-trading/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and Imports

In [ ]:
# Data collection
from yahoo_fin import stock_info

# Data preparation
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## Data Collection

In [ ]:
# Collect TQQQ historical data from yahoo_fin library
raw_df = stock_info.get_data(ticker="TQQQ", start_date="2011-01-01", end_date="2023-12-31", index_as_date=False)

## Data Exploration

In [ ]:
# Data structure
raw_df

,date,open,high,low,close,adjclose,volume,ticker
0,2011-01-03,0.790677,0.820885,0.790573,0.805000,0.785481,66758400,TQQQ
1,2011-01-04,0.815104,0.815313,0.788385,0.803750,0.784261,65760000,TQQQ
2,2011-01-05,0.795781,0.824063,0.794740,0.823698,0.803726,61440000,TQQQ
3,2011-01-06,0.826615,0.833073,0.821510,0.831146,0.810993,52262400,TQQQ
4,2011-01-07,0.835365,0.836771,0.805156,0.829219,0.809113,76204800,TQQQ
...,...,...,...,...,...,...,...,...
3265,2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200,TQQQ
3266,2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600,TQQQ
3267,2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800,TQQQ
3268,2023-12-28,51.919998,52.009998,51.270000,51.410000,51.410000,42884100,TQQQ


In [ ]:
# Check for missing values
raw_df.isnull().sum()

date        0
open        0
high        0
low         0
close       0
adjclose    0
volume      0
ticker      0
dtype: int64

In [ ]:
# Check for duplicates
raw_df.duplicated().sum()

0

## Data Processing & Preparation

In [ ]:
# Clean data
raw_df = raw_df.drop(columns="ticker")
raw_df

,date,open,high,low,close,adjclose,volume
0,2011-01-03,0.790677,0.820885,0.790573,0.805000,0.785481,66758400
1,2011-01-04,0.815104,0.815313,0.788385,0.803750,0.784261,65760000
2,2011-01-05,0.795781,0.824063,0.794740,0.823698,0.803726,61440000
3,2011-01-06,0.826615,0.833073,0.821510,0.831146,0.810993,52262400
4,2011-01-07,0.835365,0.836771,0.805156,0.829219,0.809113,76204800
...,...,...,...,...,...,...,...
3265,2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200
3266,2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600
3267,2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800
3268,2023-12-28,51.919998,52.009998,51.270000,51.410000,51.410000,42884100


In [ ]:
raw_df = raw_df.dropna()

In [ ]:
# Set index as the date
raw_df = raw_df.set_index("date")
raw_df

,open,high,low,close,adjclose,volume
date,,,,,,
2011-01-03,0.790677,0.820885,0.790573,0.805000,0.785481,66758400
2011-01-04,0.815104,0.815313,0.788385,0.803750,0.784261,65760000
2011-01-05,0.795781,0.824063,0.794740,0.823698,0.803726,61440000
2011-01-06,0.826615,0.833073,0.821510,0.831146,0.810993,52262400
2011-01-07,0.835365,0.836771,0.805156,0.829219,0.809113,76204800
...,...,...,...,...,...,...
2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200
2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600
2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800


In [ ]:
# Process data - calculate and include stock measurements
# Functions needed for calculations
def _weighted_moving_avg(close_series: pd.Series, window: int) -> pd.Series:
    # Define weights
    weights = np.arange(1, window + 1)
    return close_series.rolling(window).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

def _hull_moving_avg(close_series: pd.Series, window: int) -> pd.Series:
    weighted_half_window = _weighted_moving_avg(close_series, window // 2)
    weighted_full_window = _weighted_moving_avg(close_series, window)
    hma_series = 2 * weighted_half_window - weighted_full_window
    hma_series = pd.Series(_weighted_moving_avg(hma_series, int(np.sqrt(window))), index=close_series.index)

    return hma_series.dropna()

In [ ]:
# Function for velocity calculations
def create_velocity_data(window: int, time_shift: int = 0) -> None:
    if time_shift == 0:
        close_series = raw_df['close']
        log_close_series = pd.Series(np.log(close_series), index=close_series.index)
        hma_series = _hull_moving_avg(log_close_series, window)
        new_series = hma_series.diff()
    else:
        new_series = pd.Series(raw_df[f"velocity_{window}w_0ts"].shift(time_shift), index=raw_df.index)
    new_series.dropna(inplace=True)
    raw_df[f"velocity_{window}w_{time_shift}ts"] = new_series

In [ ]:
windows = [16, 32, 64]
time_shifts = [2, 4, 6, 8, 10]

for window in windows:
    # Add velocity to df_to_process
    create_velocity_data(window)
    for time_shift in time_shifts:
        create_velocity_data(window, time_shift)

In [ ]:
raw_df.tail()

,open,high,low,close,adjclose,volume,velocity_16w_0ts,velocity_16w_2ts,velocity_16w_4ts,velocity_16w_6ts,...,velocity_32w_4ts,velocity_32w_6ts,velocity_32w_8ts,velocity_32w_10ts,velocity_64w_0ts,velocity_64w_2ts,velocity_64w_4ts,velocity_64w_6ts,velocity_64w_8ts,velocity_64w_10ts
date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200,0.002776,0.011531,0.024232,0.025285,...,0.012929,0.008598,0.001397,-0.004954,0.009487,0.010824,0.011916,0.011423,0.010181,0.009468
2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600,0.002347,0.006711,0.021957,0.025015,...,0.014702,0.010762,0.005871,-0.002112,0.009103,0.010176,0.012034,0.011656,0.011018,0.009694
2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800,0.003471,0.002776,0.011531,0.024232,...,0.012371,0.012929,0.008598,0.001397,0.008772,0.009487,0.010824,0.011916,0.011423,0.010181
2023-12-28,51.919998,52.009998,51.270000,51.410000,51.410000,42884100,0.002772,0.002347,0.006711,0.021957,...,0.012022,0.014702,0.010762,0.005871,0.008280,0.009103,0.010176,0.012034,0.011656,0.011018
2023-12-29,51.380001,51.520000,50.000000,50.700001,50.700001,66986200,0.000014,0.003471,0.002776,0.011531,...,0.011447,0.012371,0.012929,0.008598,0.007477,0.008772,0.009487,0.010824,0.011916,0.011423


In [ ]:
# Function for acceleration calculations
def _create_acceleration_data(window: int, time_shift: int = 0):
    if time_shift == 0:
        new_series = pd.Series(raw_df[f"velocity_{window}w_{time_shift}ts"].diff(), index=raw_df.index)
    else:
        new_series = pd.Series(raw_df[f"acceleration_{window}w_0ts"].shift(time_shift), index=raw_df.index)
    new_series.dropna(inplace=True)
    raw_df[f"acceleration_{window}w_{time_shift}ts"] = new_series

In [ ]:
for window in windows:
    _create_acceleration_data(window)
    for time_shift in time_shifts:
        _create_acceleration_data(window, time_shift)

In [ ]:
raw_df.tail()

,open,high,low,close,adjclose,volume,velocity_16w_0ts,velocity_16w_2ts,velocity_16w_4ts,velocity_16w_6ts,...,acceleration_32w_4ts,acceleration_32w_6ts,acceleration_32w_8ts,acceleration_32w_10ts,acceleration_64w_0ts,acceleration_64w_2ts,acceleration_64w_4ts,acceleration_64w_6ts,acceleration_64w_8ts,acceleration_64w_10ts
date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200,0.002776,0.011531,0.024232,0.025285,...,0.002167,0.002726,0.003509,0.001330,-0.000688,-0.001210,0.000260,0.000406,0.000487,-0.000206
2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600,0.002347,0.006711,0.021957,0.025015,...,0.001773,0.002165,0.004474,0.002842,-0.000385,-0.000648,0.000118,0.000233,0.000836,0.000226
2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800,0.003471,0.002776,0.011531,0.024232,...,-0.002331,0.002167,0.002726,0.003509,-0.000331,-0.000688,-0.001210,0.000260,0.000406,0.000487
2023-12-28,51.919998,52.009998,51.270000,51.410000,51.410000,42884100,0.002772,0.002347,0.006711,0.021957,...,-0.000350,0.001773,0.002165,0.004474,-0.000491,-0.000385,-0.000648,0.000118,0.000233,0.000836
2023-12-29,51.380001,51.520000,50.000000,50.700001,50.700001,66986200,0.000014,0.003471,0.002776,0.011531,...,-0.000574,-0.002331,0.002167,0.002726,-0.000803,-0.000331,-0.000688,-0.001210,0.000260,0.000406


In [ ]:
def _create_avg_true_range_data(window: int, time_shift: int = 0):
    if time_shift == 0:
        data_index = raw_df.index
        high_series = raw_df['high']
        low_series = raw_df['low']
        close_prev_series = raw_df['close'].shift(1)

        true_range = (
            pd.DataFrame({
                'h_l': high_series - low_series,
                'h_c_prev': abs(high_series - close_prev_series),
                'l_c_prev': abs(low_series - close_prev_series)
            }, index=data_index)
            .max(axis=1)
        )
        true_range_series = _hull_moving_avg(true_range, window)
    else:
        true_range_series = raw_df[f"atr_{window}w_0ts"].shift(time_shift)
    raw_df[f"atr_{window}w_{time_shift}ts"] = true_range_series

In [ ]:
for window in windows:
    _create_avg_true_range_data(window)
    for time_shift in time_shifts:
        _create_avg_true_range_data(window, time_shift)

In [ ]:
raw_df.tail()

,open,high,low,close,adjclose,volume,velocity_16w_0ts,velocity_16w_2ts,velocity_16w_4ts,velocity_16w_6ts,...,atr_32w_4ts,atr_32w_6ts,atr_32w_8ts,atr_32w_10ts,atr_64w_0ts,atr_64w_2ts,atr_64w_4ts,atr_64w_6ts,atr_64w_8ts,atr_64w_10ts
date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,50.599998,50.939999,49.639999,50.349998,50.349998,66114200,0.002776,0.011531,0.024232,0.025285,...,1.764065,1.739792,1.635100,1.568927,1.538212,1.504341,1.492680,1.468878,1.422921,1.397094
2023-12-26,50.639999,51.509998,50.619999,51.270000,51.270000,43685600,0.002347,0.006711,0.021957,0.025015,...,1.711046,1.762311,1.675607,1.606952,1.540090,1.525608,1.484582,1.484214,1.441433,1.411445
2023-12-27,51.349998,51.660000,51.020000,51.540001,51.540001,55747800,0.003471,0.002776,0.011531,0.024232,...,1.731546,1.764065,1.739792,1.635100,1.522899,1.538212,1.504341,1.492680,1.468878,1.422921
2023-12-28,51.919998,52.009998,51.270000,51.410000,51.410000,42884100,0.002772,0.002347,0.006711,0.021957,...,1.745776,1.711046,1.762311,1.675607,1.493345,1.540090,1.525608,1.484582,1.484214,1.441433
2023-12-29,51.380001,51.520000,50.000000,50.700001,50.700001,66986200,0.000014,0.003471,0.002776,0.011531,...,1.736588,1.731546,1.764065,1.739792,1.469515,1.522899,1.538212,1.504341,1.492680,1.468878


In [ ]:
# Normalize all data points
def _normalize_data(df):
    # Extract the date column
    date_column = df.index
    # Convert the DataFrame to a NumPy array
    data_array = df.values

    # Normalize the data except for the date column
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data_array)

    # Recreate the DataFrame with the normalized data
    df = pd.DataFrame(normalized_data, index=date_column, columns=df.columns)
    return df

In [ ]:
prepared_df = _normalize_data(raw_df)

In [ ]:
prepared_df.head()

,open,high,low,close,adjclose,volume,velocity_16w_0ts,velocity_16w_2ts,velocity_16w_4ts,velocity_16w_6ts,...,atr_32w_4ts,atr_32w_6ts,atr_32w_8ts,atr_32w_10ts,atr_64w_0ts,atr_64w_2ts,atr_64w_4ts,atr_64w_6ts,atr_64w_8ts,atr_64w_10ts
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,0.002694,0.002588,0.002947,0.002948,0.002940,0.099622,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,0.002969,0.002527,0.002921,0.002934,0.002926,0.097751,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,0.002752,0.002623,0.002994,0.003160,0.003152,0.089655,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,0.003098,0.002722,0.003302,0.003245,0.003237,0.072454,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-07,0.003196,0.002763,0.003114,0.003223,0.003215,0.117326,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Backfill missing values
def backfill_data(df):
    for column in df.columns[4:]:
        df[column] = df[column].interpolate(method='bfill')
    return df

In [ ]:
ready_data = backfill_data(prepared_df)

/tmp/ipykernel_156459/657169231.py:4: FutureWarning: Series.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column] = df[column].interpolate(method='bfill')


In [ ]:
ready_data.head()

,open,high,low,close,adjclose,volume,velocity_16w_0ts,velocity_16w_2ts,velocity_16w_4ts,velocity_16w_6ts,...,atr_32w_4ts,atr_32w_6ts,atr_32w_8ts,atr_32w_10ts,atr_64w_0ts,atr_64w_2ts,atr_64w_4ts,atr_64w_6ts,atr_64w_8ts,atr_64w_10ts
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,0.002694,0.002588,0.002947,0.002948,0.002940,0.099622,0.607,0.607,0.607,0.607,...,0.000818,0.000818,0.000818,0.000818,0.001998,0.001998,0.001998,0.001998,0.001998,0.001998
2011-01-04,0.002969,0.002527,0.002921,0.002934,0.002926,0.097751,0.607,0.607,0.607,0.607,...,0.000818,0.000818,0.000818,0.000818,0.001998,0.001998,0.001998,0.001998,0.001998,0.001998
2011-01-05,0.002752,0.002623,0.002994,0.003160,0.003152,0.089655,0.607,0.607,0.607,0.607,...,0.000818,0.000818,0.000818,0.000818,0.001998,0.001998,0.001998,0.001998,0.001998,0.001998
2011-01-06,0.003098,0.002722,0.003302,0.003245,0.003237,0.072454,0.607,0.607,0.607,0.607,...,0.000818,0.000818,0.000818,0.000818,0.001998,0.001998,0.001998,0.001998,0.001998,0.001998
2011-01-07,0.003196,0.002763,0.003114,0.003223,0.003215,0.117326,0.607,0.607,0.607,0.607,...,0.000818,0.000818,0.000818,0.000818,0.001998,0.001998,0.001998,0.001998,0.001998,0.001998
